In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eeg-eye-state-dataset/train.csv
/kaggle/input/eeg-eye-state-dataset/test.csv


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("syednazmussakib/eeg-eye-state-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/eeg-eye-state-dataset


In [3]:
train=pd.read_csv("/kaggle/input/eeg-eye-state-dataset/train.csv")

In [4]:
train.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,target,id
0,4313.33,4008.72,4261.03,4090.26,4342.56,4626.15,4078.46,4633.85,4237.95,4274.87,4236.41,4304.62,4633.33,4386.67,0,1
1,4316.92,4016.92,4262.56,4101.03,4344.62,4632.31,4082.05,4643.08,4248.21,4286.15,4250.26,4311.79,4642.56,4390.77,0,2
2,4321.03,4018.46,4262.56,4107.69,4345.13,4639.49,4088.72,4645.64,4251.79,4284.10,4248.21,4312.82,4640.51,4387.69,0,3
3,4318.97,4009.23,4265.13,4102.56,4339.49,4637.95,4090.26,4649.23,4255.90,4283.08,4240.00,4311.79,4635.90,4369.23,0,4
4,4305.13,3997.95,4265.64,4093.85,4331.79,4635.90,4088.72,4650.26,4252.31,4287.69,4237.44,4308.21,4628.72,4355.38,0,5


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AF3     9980 non-null   float64
 1   F7      9980 non-null   float64
 2   F3      9980 non-null   float64
 3   FC5     9980 non-null   float64
 4   T7      9980 non-null   float64
 5   P7      9980 non-null   float64
 6   O1      9980 non-null   float64
 7   O2      9980 non-null   float64
 8   P8      9980 non-null   float64
 9   T8      9980 non-null   float64
 10  FC6     9980 non-null   float64
 11  F4      9980 non-null   float64
 12  F8      9980 non-null   float64
 13  AF4     9980 non-null   float64
 14  target  9980 non-null   int64  
 15  id      9980 non-null   int64  
dtypes: float64(14), int64(2)
memory usage: 1.2 MB


In [6]:
train.shape

(9980, 16)

In [7]:
train.isna().sum()

AF3       0
F7        0
F3        0
FC5       0
T7        0
P7        0
O1        0
O2        0
P8        0
T8        0
FC6       0
F4        0
F8        0
AF4       0
target    0
id        0
dtype: int64

In [8]:
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix

In [9]:
cols = list(train.columns)
cols

['AF3',
 'F7',
 'F3',
 'FC5',
 'T7',
 'P7',
 'O1',
 'O2',
 'P8',
 'T8',
 'FC6',
 'F4',
 'F8',
 'AF4',
 'target',
 'id']

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import seaborn as sns

In [11]:

#fig, axs = plt.subplots(1, len(cols), figsize=(5*len(cols), 3))

#for i, column in enumerate(cols):
   ##_ = fig.tight_layout()

In [12]:
test=pd.read_csv("/kaggle/input/eeg-eye-state-dataset/test.csv")

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
y=train["target"]
train.pop("target")
X=train

In [14]:
X.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,id
0,4313.33,4008.72,4261.03,4090.26,4342.56,4626.15,4078.46,4633.85,4237.95,4274.87,4236.41,4304.62,4633.33,4386.67,1
1,4316.92,4016.92,4262.56,4101.03,4344.62,4632.31,4082.05,4643.08,4248.21,4286.15,4250.26,4311.79,4642.56,4390.77,2
2,4321.03,4018.46,4262.56,4107.69,4345.13,4639.49,4088.72,4645.64,4251.79,4284.10,4248.21,4312.82,4640.51,4387.69,3
3,4318.97,4009.23,4265.13,4102.56,4339.49,4637.95,4090.26,4649.23,4255.90,4283.08,4240.00,4311.79,4635.90,4369.23,4
4,4305.13,3997.95,4265.64,4093.85,4331.79,4635.90,4088.72,4650.26,4252.31,4287.69,4237.44,4308.21,4628.72,4355.38,5


In [15]:
X.columns

Index(['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6',
       'F4', 'F8', 'AF4', 'id'],
      dtype='object')

In [16]:
test.columns

Index(['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6',
       'F4', 'F8', 'AF4', 'target', 'id'],
      dtype='object')

In [17]:
X_transf=sc.fit_transform(X)


In [18]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(X_transf,y,test_size=.2,random_state=0)

In [19]:
linear_reg = LinearRegression()
svr_reg = SVR(kernel='sigmoid') # You can choose different kernels if needed
xgb_reg = XGBRegressor(random_state=42)
rf_reg = RandomForestRegressor(random_state=42)

In [20]:
scores = cross_val_score(linear_reg,
                         X=xtrain,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: -24.333 +/- 73.929


In [21]:
scores = cross_val_score(xgb_reg,
                         X=xtrain,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.979 +/- 0.006


In [22]:
scores = cross_val_score(rf_reg,
                         X=xtrain,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.981 +/- 0.007
